# TP : Analyse de sentiments dans les critiques de films

## Objectifs

1. Implémenter une manière simple de représenter des données textuelles
2. Implémenter un modèle d'apprentissage statistique basique
3. Utiliser ces représentations et ce modèle pour une tâche d'analyse de sentiments
4. Tenter d'améliorer les résultats avec des outils venus du traitement automatique du langage
5. Comparer les résultats avec une l'implémentation de Scikit-Learn, et avec d'autres méthodes de représentation ou d'apprentissage.

## Dépendances nécessaires

Pour les objectifs 4. et 5., on aura besoin des packages suivants:
- The Machine Learning API Scikit-learn : http://scikit-learn.org/stable/install.html
- The Natural Language Toolkit : http://www.nltk.org/install.html

Les deux sont disponibles avec Anaconda: https://anaconda.org/anaconda/nltk et https://anaconda.org/anaconda/scikit-learn

In [1]:
import os.path as op
import re 
import numpy as np

## Charger les données

Extraire les données et placer le dossier 'data' dans le même dossier que le notebook.

On récupère les données textuelles dans la variable *texts*

On récupère les labels dans la variable $y$ qui en contient *len(texts)* : $0$ indique que la critique correspondante est négative tandis que $1$ qu'elle est positive.

In [2]:
from glob import glob
# We get the files from the path: ./data/imdb1/neg for negative reviews, and ./data/imdb1/pos for positive reviews
filenames_neg = sorted(glob(op.join('.', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('.', 'data', 'imdb1', 'pos', '*.txt')))

# Each files contains a review that consists in one line of text: we put this string in two lists, that we concatenate
texts_neg = [open(f, encoding="utf8").read() for f in filenames_neg]
texts_pos = [open(f, encoding="utf8").read() for f in filenames_pos]
texts = texts_neg + texts_pos

# The first half of the elements of the list are string of negative reviews, and the second half positive ones
# We create the labels, as an array of [1,len(texts)], filled with 1, and change the first half to 0
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

2000 documents


In [3]:
# This number of documents may be high for most computers: we can select a fraction of them (here, one in k)
# Use an even number to keep the same number of positive and negative reviews
k = 10
texts_reduced = texts[0::k]
y_reduced = y [0::k]

print('Nombre de documents:', len(texts_reduced))

Nombre de documents: 200


## Idée principale

On dispose d'une critique étant en fait une liste de mots $s = (w_1, ..., w_N)$, et l'on cherche à trouver la classe associée $c$ - qui dans notre cas, peut-être $c = 0$ ou $c = 1$. L'objectif est donc de trouver pour chaque critique $s$ la classe $\hat{c}$ maximisant la probabilité conditionelle **$P(c|s)$** : 

$$\hat{c} = \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)}$$

**Hypothèse : P(s) est constante pour chaque classe** :

$$\hat{c} = \underset{c}{\mathrm{argmax}}\,\frac{P(s|c)P(c)}{P(s)} = \underset{c}{\mathrm{argmax}}\,P(s|c)P(c)$$

**Hypothèse naïve : les différentes variables (mots) d'une critique sont indépendantes entre elles** : 

$$P(s|c) = P(w_1, ..., w_N|c)=\Pi_{i=1..N}P(w_i|c)$$

On va donc pouvoir se servir des critiques annotées à notre disposition pour **estimer les probabilités $P(w|c)$ pour chaque mot $w$ étant donné les deux classes $c$**. Ces critiques vont nous permettre d'apprendre à évaluer la "compatibilité" entre les mots et classes.

## Vue générale

### Entraînement: Estimer les probabilités

Pour chaque mot $w$ du vocabulaire $V$, $P(w|c)$ est le nombre d'occurences de $w$ dans une critique ayant pour classe $c$, divisé par le nombre total d'occurences dans $c$. Si on note $T(w,c)$ ce nombre d'occurences, on obtient:

$$P(w|c) = \text{Fréquence de }w\text{ dans }c = \frac{T(w,c)}{\sum_{w' \in V} T(w',c)}$$

### Test: Calcul des scores

Pour faciliter les calculs et éviter les erreurs d'*underflow* et d'approximation, on utilise le "log-sum trick", et on passe l'équation en log-probabilités : 

$$\hat{c} =  \underset{c}{\mathrm{argmax}}\, P(c|s) = \underset{c}{\mathrm{argmax}}\, \left[ \mathrm{log}(P(c)) + \sum_{i=1..N}log(P(w_i|c)) \right]$$

### Laplace smoothing (Lissage)

Un mot qui n'apparaît pas dans un document a une probabilité nulle: cela va poser problème avec le logarithme. On garde donc une toute petite partie de la masse de probabilité qu'on redistribue avec le *Laplace smoothing*: 

$$P(w|c) = \frac{T(w,c) + 1}{\sum_{w' \in V} T(w',c) + 1}$$

Il existe d'autre méthodes de lissage, en général adaptées à d'autres applications plus complexes. 

## Représentation adaptée des documents

Notre modèle statistique, comme la plupart des modèles appliqués aux données textuelles, utilise les comptes d'occurences de mots dans un document. Ainsi, une manière très pratique de représenter un document est d'utiliser un vecteur "Bag-of-Words" (BoW), contenant les comptes de chaque mot (indifférement de leur ordre d'apparition) dans le document. 

Si on considère l'ensemble de tous les mots apparaissant dans nos $T$ documents d'apprentissage, que l'on note $V$ (Vocabulaire), on peut créer **un index**, qui est une bijection associant à chaque mot $w$ un entier, qui sera sa position dans $V$. 

Ainsi, pour un document extrait d'un ensemble de documents contenant $|V|$ mots différents, une représentation BoW sera un vecteur de taille $|V|$, dont la valeur à l'indice d'un mot $w$ sera son nombre d'occurences dans le document. 

On peut utiliser la classe **CountVectorizer** de scikit-learn pour mieux comprendre:

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin

In [5]:
corpus = ['I walked down down the boulevard',
          'I walked down the avenue',
          'I ran down the boulevard',
          'I walk down the city',
          'I walk down the the avenue']
vectorizer = CountVectorizer()

Bow = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
Bow.toarray()

['avenue', 'boulevard', 'city', 'down', 'ran', 'the', 'walk', 'walked']


array([[0, 1, 0, 2, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 2, 1, 0]], dtype=int64)

On affiche d'abord la liste contenant les mots ordonnés selon leur indice (On note que les mots de 2 caractères ou moins ne sont pas pris en compte).

## Détail: entraînement

L'idée est d'extraire le nombre d'occurences $T(w,c)$ de chaque mot $w$ pour chaque classe $c$, ce qui permettra de calculer la matrice de probabilités conditionelles $\pmb{P}$ telle que: $$\pmb{P}_{w,c} = P(w|c)$$

Notons que le nombre d'occurences $T(w,c)$ peut être obtenu facilement à partir des représentations BoW de l'ensemble des documents.

### Procédure:
<img src="algo_train.png" alt="Drawing" style="width: 700px;"/>

## Détail: test

Nous connaissons maintenant les probabilités conditionelles données par la matrice $\pmb{P}$. 
Il faut maintenant obtenir $P(s|c)$ pour le document courant. Cette quantité s'obtient à l'aide d'un calcul simple impliquant la représentation BoW du document et $\pmb{P}$.

### Procédure:
<img src="algo_test.png" alt="Drawing" style="width: 700px;"/>

## Preprocessing du texte: obtenir les représentations BoW

D'abord, il faut transformer les critiques sous forme de strings en une liste de mots. La tactique la plus simple consiste à diviser le string suivant les espaces, avec la commande:
```text.split()```

Mais il faut aussi faire attention à enlever les caractères particuliers qui pourraient ne pas avoir été nettoyés (comme les balises HTML si on a obtenu les données à partir de pages web). Puisque l'on va compter les mots, il faudra construire une liste des mots apparaissant dans nos données. Dans notre cas, on aimerait réduire cette liste et l'uniformiser (ignorer les majuscules, la ponctuation, et les mots les plus courts). 
On va donc utiliser une fonction adaptée à nos besoins - mais c'est un travail qu'il n'est en général pas nécessaire de faire, puisqu'il existe de nombreux outils déjà adaptés à la plupart des cas de figures. 
Pour le nettoyage du texte, il existe de nombreux scripts, basés sur différents outils (expressions régulières, par exemple) qui permettent de préparer des données. La division du texte en mots et la gestion de la ponctuation est gérée lors d'une étape appellée *tokenization*; si besoin, un package python comme le NLTK contient de nombreux *tokenizers* différents.

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jean/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# We might want to clean the file with various strategies:
def clean_and_tokenize(text):
    """
    Cleaning a document with:
        - Lowercase        
        - Removing numbers with regular expressions
        - Removing punctuation with regular expressions
        - Removing other artifacts
    And separate the document into words by simply splitting at spaces
    Params:
        text (string): a sentence or a document
    Returns:
        tokens (list of strings): the list of tokens (word units) forming the document
    """        
    # Lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r"[0-9]+", "", text)
    # Remove punctuation
    REMOVE_PUNCT = re.compile("[.;:!\'?,\"()\[\]]")
    text = REMOVE_PUNCT.sub("", text)
    # Remove small words (1 and 2 characters)
    text = re.sub(r"\b\w{1,2}\b", "", text)
    # Remove HTML artifacts specific to the corpus we're going to work with
    REPLACE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    text = REPLACE_HTML.sub(" ", text)
    
    tokens = text.split()        
    return tokens

# Or we might want to use an already-implemented tool. The NLTK package has a lot of very useful text processing tools, among them various tokenizers
# Careful, NLTK was the first well-documented NLP package, but it might be outdated for some uses. Check the documentation !
from nltk.tokenize import word_tokenize

corpus_raw = "I walked down down the boulevard. I walked down the avenue. I ran down the boulevard. I walk down the city. I walk down the the avenue."
print(clean_and_tokenize(corpus_raw))
print(word_tokenize(corpus_raw))

['walked', 'down', 'down', 'the', 'boulevard', 'walked', 'down', 'the', 'avenue', 'ran', 'down', 'the', 'boulevard', 'walk', 'down', 'the', 'city', 'walk', 'down', 'the', 'the', 'avenue']
['I', 'walked', 'down', 'down', 'the', 'boulevard', '.', 'I', 'walked', 'down', 'the', 'avenue', '.', 'I', 'ran', 'down', 'the', 'boulevard', '.', 'I', 'walk', 'down', 'the', 'city', '.', 'I', 'walk', 'down', 'the', 'the', 'avenue', '.']


Fonction **à compléter**. Elle prend en entrée une liste de document (chacun sous la forme d'un string) et renvoie, comme dans l'exemple utilisant ```CountVectorizer```:
- Un vocabulaire qui associe à chaque mot rencontré un index
- Une matrice, dont les lignes représentent les documents et les colonnes les mots indexés par le vocabulaire. En position $(i,j)$, on devra avoir le nombre d'occurence du mot $j$ dans le document $i$.

Le vocabulaire, qui était sous la forme d'une *liste* dans l'exemple précédent, pourra être renvoyé sous forme de *dictionnaire* dont les clés sont les mots et les valeurs les indices. Puisque le vocabulaire recense les mots du corpus sans se soucier de leur nombre d'occurences, on pourra le constituer à l'aide d'un ensemble (```set``` en python). 
On pourra bien sur utiliser la fonction ```clean_and_tokenize``` pour transformer les strings en liste de mots. 
##### Quelques pointeurs pour les débutants en Python : 

- ```my_list.append(value)``` : put the variable ```value``` at the end of the list ```my_list```

-  ```words = set()``` : create a set, which is a list of unique values 

- ```words.union(my_list)``` : extend the set ```words```

- ```dict(zip(keys, values)))``` : create a dictionnary

- ```for k, text in enumerate(texts)``` : syntax for a loop with the index, ```texts``` begin a list (of texts !)

- ```len(my-list)``` : length of the list ```my_list```


In [8]:
corpus

['I walked down down the boulevard',
 'I walked down the avenue',
 'I ran down the boulevard',
 'I walk down the city',
 'I walk down the the avenue']

In [9]:
def count_words(texts):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts
    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
    """
    clean=[]
    for text in texts:
        clean.append(clean_and_tokenize(text))
        
    flatten = []
    for sublist in clean:
        for item in sublist:
            flatten.append(item)
    
    myset = sorted(set(flatten))
    
    keys = range(0,len(myset))
    
    vocabulary = dict(zip(myset,keys))
    
    counts = np.zeros((len(clean),len(vocabulary)))            
    
    for k,text in enumerate(clean):
        for word in text:
            if word in vocabulary:
                counts[k,vocabulary[word]] += 1
                
                
    
    return vocabulary,counts

In [10]:
Voc,X= count_words(corpus)
print(Voc)
print(X)


{'avenue': 0, 'boulevard': 1, 'city': 2, 'down': 3, 'ran': 4, 'the': 5, 'walk': 6, 'walked': 7}
[[0. 1. 0. 2. 0. 1. 0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 1.]
 [0. 1. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 1. 1. 0.]
 [1. 0. 0. 1. 0. 2. 1. 0.]]


## Naïve Bayes 

Classe vide : fonctions **à compléter** 
```python
def fit(self, X, y)
``` 
**Entraînement** : va apprendre un modèle statistique basés sur les représentations $X$ correspondant aux labels $y$.
$X$ représente donc ici des représentations obtenues en sortie de count_words. On complète la fonction à l'aide de la procédure détaillée plus haut. Si il est possible de la suivre à la lettre, les représentations que l'on utilise nous permettre d'être bien plus efficace et d'éviter d'utiliser des boucles !


Note: le lissage, effectué à la ligne $10$, ne se fait pas nécessairement avec un $1$, mais peut se faire avec une valeur positive $\alpha$, qu'on peut implémenter comme argument de la classe ```NB```.

```python
def predict(self, X)
```
**Testing** : va renvoyer les labels prédits par le modèle pour d'autres représentations $X$.



Pour faciliter la procédure, on prendra la moitié de la matrice $X$ obtenue plus haut pour entraîner le modèle, et l'autre moitié pour l'évaluer. **Important**: cette façon de procéder n'est pas réaliste: en général, on ne dispose que des données d'entraînement au moment de créer le vocabulaire et d'entraîner le modèle. Ainsi, il est possible que les données d'évaluation contiennent des mots *inconnus*. C'est quelque chose qu'on peut traiter facilement en dédiant un indice à tous les mots rencontrés qui ne sont pas contenus dans le vocabulaire - mais il existe de nombreuses méthodes plus complexes pour réussir à utiliser à bon escient ces mots que le modèle n'a pas rencontré à l'entraînement. 

##### Quelques pointeurs pour les débutants en Python : 

Utiliser l'API Numpy pour travailler avec des tenseurs


- ```X.shape``` : for a ```numpy.array```, return the dimension of the tensor

- ```np.zeros((dim_1, dim_2,...))``` : create a tensor filled with zeros

- ```np.sum(X, axis = n)``` : sum the tensor over the axis n

- ```np.mean(X, axis = n)```

- ```np.argmax(X, axis = n)```

- ```np.log(X)```

- ```np.dot(X_1, X_1)``` : Matrix multiplication

In [11]:
class NB(BaseEstimator, ClassifierMixin):
    # Les arguments de classe permettent l'héritage de classes de sklearn
    def __init__(self, alpha=1.0):
        # alpha est un paramètre pour le lissage: il correspond à la valeur ligne 10 de l'algorithme d'entraînement
        # Dans l'algorithme d'entraînement, et comme valeur par défaut, on utilise alpha = 1
        self.alpha = alpha

    def fit(self, X, y):
        
        # sort data into classes
        Xy0 = X[y == 0]
        Xy1 = X[y == 1]
        
        # calculate priors
        priory0 = len(Xy0) / len(X)
        priory1 = len(Xy1) / len(X)
        
        self.prior = np.array([priory0,priory1])
        
        cond_prob = np.zeros((X.shape[1],2))
        
        for i in range (len(X[0])):
            cond_prob[i,0] = (np.sum(Xy0[:,i])+1)/(np.sum(Xy0)+len(X[0]))
            cond_prob[i,1] = (np.sum(Xy1[:,i])+1)/(np.sum(Xy1)+len(X[0]))
              
        self.cond = cond_prob
                                                   
        return self

    def predict(self, X):
        
        score = np.zeros((2,X.shape[0]))
        
        
        score[0,:] = np.log(self.prior[0])
        score[0,:] += X @ np.log(self.cond[:,0])
        
        score[1,:] = np.log(self.prior[1])
        score[1,:] += X @ np.log(self.cond[:,1])

        result = np.argmax(score,axis=0)
        
        return result

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

## Expérimentation

On utilise la moitié des données pour l'entraînement, l'autre pour tester le modèle.

In [12]:
voc, X = count_words(texts)

In [14]:
nb = NB()
nb.fit(X[::2], y[::2])
print(nb.prior.shape)
print(nb.cond.shape)
print(nb.score(X[1::2], y[1::2]))

(2,)
(41354, 2)
0.821


## Cross-validation 

Avec la fonction *cross_val_score* de scikit-learn

In [15]:
scores = cross_val_score(nb, X, y, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.8099999999999999 (std 0.010839741694339374)


## Evaluer les performances: 

**Quelles sont les points forts et les points faibles de ce système ? Comment y remédier ?**

# Pour aller plus loin: 

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


## Scikit-learn

### Améliorer les représentations

On utilise la function 
```python
CountVectorizer
``` 
de scikit-learn pour constituer notre corpus. Elle nous permettra d'améliorer facilement nos représentations BoW.

#### Tf-idf:

Il s'agit du produit de la fréquence du terme (TF) et de sa fréquence inverse dans les documents (IDF).
Cette méthode est habituellement utilisée pour extraire l'importance d'un terme $i$ dans un document $j$ relativement au reste du corpus, à partir d'une matrice d'occurences $mots \times documents$. Ainsi, pour une matrice $\mathbf{T}$ de $|V|$ termes et $D$ documents:
$$\text{TF}(T, w, d) = \frac{T_{w,d}}{\sum_{w'=1}^{|V|} T_{w',d}} $$

$$\text{IDF}(T, w) = \log\left(\frac{D}{|\{d : T_{w,d} > 0\}|}\right)$$

$$\text{TF-IDF}(T, w, d) = \text{TF}(X, w, d) \cdot \text{IDF}(T, w)$$

On peut l'adapter à notre cas en considérant que le contexte du deuxième mot est le document. Cependant, TF-IDF est généralement plus adaptée aux matrices peu denses, puisque cette mesure pénalisera les termes qui apparaissent dans une grande partie des documents. 
    
#### Ne pas prendre en compte les mots trop fréquents:

On peut utiliser l'option 
```python
max_df=1.0
```
pour modifier la quantité de mots pris en compte. 

#### Essayer différentes granularités:

Plutôt que de simplement compter les mots, on peut compter les séquences de mots - de taille limitée, bien sur. 
On appelle une séquence de $n$ mots un $n$-gram: essayons d'utiliser les 2 et 3-grams (bi- et trigrams).
On peut aussi tenter d'utiliser les séquences de caractères à la place de séquences de mots.

On s'intéressera aux options 
```python
analyzer='word'
```
et 
```python
ngram_range=(1, 2)
```
que l'on changera pour modifier la granularité. 

In [33]:
## On peut définir une pipeline que l'on modifiera pour expérimenter.

pipeline_base = Pipeline([
    ('vect', CountVectorizer(analyzer='word', stop_words=None)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(pipeline_base, texts, y, cv=5)
print("Classification score: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_tf_idf = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None)),
        ('tfid', TfidfTransformer()),
        ('clf', MultinomialNB())
])
scores = cross_val_score(pipeline_tf_idf, texts, y, cv=5)
print("Classification score tf-idf: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_maxdf = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None,max_df=1.0)),
        ('clf', MultinomialNB())
        
])
scores = cross_val_score(pipeline_maxdf, texts, y, cv=5)
print("Classification score sans mots fréquents: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_bigram = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None,ngram_range=(1, 2))),
        ('clf', MultinomialNB())
        
])
scores = cross_val_score(pipeline_bigram, texts, y, cv=5)
print("Classification score bigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_trigram = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None,ngram_range=(1, 3))),
        ('clf', MultinomialNB())
        
])
scores = cross_val_score(pipeline_trigram, texts, y, cv=5)
print("Classification score trigram: %s (std %s)",(np.mean(scores), np.std(scores)))

pipeline_char = Pipeline([
        ('vect', CountVectorizer(analyzer='char', stop_words=None)),
        ('clf', MultinomialNB())
 
])
scores = cross_val_score(pipeline_char, texts, y, cv=5)
print("Classification score char: %s (std %s)",(np.mean(scores), np.std(scores)))

Classification score: %s (std %s) (0.812, 0.012884098726725092)
Classification score tf-idf: %s (std %s) (0.8125, 0.010488088481701477)
Classification score sans mots fréquents: %s (std %s) (0.812, 0.012884098726725092)
Classification score bigram: %s (std %s) (0.8305, 0.009273618495495711)
Classification score trigram: %s (std %s) (0.8225, 0.017320508075688742)
Classification score char: %s (std %s) (0.6094999999999999, 0.018261982367749664)


### Natural Language Toolkit (NLTK)

### Stemming 

Permet de revenir à la racine d'un mot: on peut ainsi grouper différents mots autour de la même racine, ce qui facilite la généralisation. Utiliser:
```python
from nltk import SnowballStemmer
```

In [18]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer("english")

#### Exemple d'utilisation:

In [19]:
words = ['singers', 'cat', 'generalization', 'philosophy', 'psychology', 'philosopher']
for word in words:
    print('word : %s ; stemmed : %s' %(word, stemmer.stem(word)))#.decode('utf-8'))))

word : singers ; stemmed : singer
word : cat ; stemmed : cat
word : generalization ; stemmed : general
word : philosophy ; stemmed : philosophi
word : psychology ; stemmed : psycholog
word : philosopher ; stemmed : philosoph


#### Transformation des données:

Classe vide : function **à compléter** 
```python
def stem(X)
``` 

In [20]:
def stem(X): 
    X_stem = []
    for word in X:
        X_stem.append(stemmer.stem(word))
    return X_stem

In [36]:
texts_stemmed = stem(texts)
voc, X = count_words(texts_stemmed)
nb = NB()

scores = cross_val_score(nb, X, y, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.8099999999999999 (std 0.010839741694339374)


### Partie du discours

Pour généraliser, on peut aussi utiliser les parties du discours (Part of Speech, POS) des mots, ce qui nous permettra  de filtrer l'information qui n'est potentiellement pas utile au modèle. On va récupérer les POS des mots à l'aide des fonctions:
```python
from nltk import pos_tag, word_tokenize
```

In [22]:
import nltk
from nltk import pos_tag, word_tokenize

#### Exemple d'utilisation:

In [23]:
import nltk
# nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

pos_tag(word_tokenize(('I am Sam')))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jean/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('I', 'PRP'), ('am', 'VBP'), ('Sam', 'NNP')]

Détails des significations des tags POS: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk

####  Transformation des données:

Classe vide : fonction **à compléter** 
```python
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB'])
``` 

Ne garder que les noms, adverbes, verbes et adjectifs pour notre modèle. 

In [26]:
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB']):
    X_pos = []
    for words in X:
        text=[]
        tokens = word_tokenize(words)
        tags = nltk.pos_tag(tokens)
        good = [t for t in tags if t[1] in good_tags]
        for i in range (len(good)):
             text.append(good[i][0])  
        X_pos.append(text)
        
    for j in range (len(X_pos)):
        X_pos[j] = ' '.join(X_pos[j])
        
    return X_pos


In [27]:
texts_POS = pos_tag_filter(texts)
voc, X = count_words(texts_POS)
nb = NB()

scores = cross_val_score(nb, X, y, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.7825000000000001 (std 0.015411035007422467)


### Stop-words 

Les "stop-words" sont les mots apparaissant fréquemment dans les données et que l'on juge non représentatifs. On les considère comme du bruit. Une liste de stop-words est disponible dans le fichier *english.stop*

In [28]:
def readFile(fileName):
    """
     * Code for reading a file.  you probably don't want to modify anything here, 
     * unless you don't like the way we segment files.
    """
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = ('\n'.join(contents)).split() 
    return result

sw = readFile('english.stop')
sw[0:50]

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated']

####  Transformation des données:

Classe vide : fonction **à compléter** 
```python
def filterStopWords(X)
``` 

In [29]:
def filterStopWords(X):
    """Filters stop words."""
    X_filtered = []
    for words in X:
        text=[]
        tokens = word_tokenize(words)
        good = [t for t in tokens if t not in sw]
        for i in range (len(good)):
             text.append(good[i])  
        X_filtered.append(text)
    
    for j in range (len(X_filtered)):
        X_filtered[j] = ' '.join(X_filtered[j])
        
    return X_filtered

In [30]:
texts_stop = filterStopWords(texts)
voc, X = count_words(texts_stop)
nb = NB()

scores = cross_val_score(nb, X, y, cv=5)
print('Score de classification: %s (std %s)' % (np.mean(scores), np.std(scores)))

Score de classification: 0.8035 (std 0.01617096162879622)


### Bonus: Utilisation d'un classifieur plus complexe ?

On peut utiliser les implémentations scikit-learn de classifieurs moins naïfs, comme la régression logistique ou les SVM. 

In [31]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [35]:
pipeline_logistic = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None,max_df=1.0)),
        ('tfid', TfidfTransformer()),
        ('clf', LinearSVC())
])
scores = cross_val_score(pipeline_logistic, texts, y, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))

pipeline_svm = Pipeline([
        ('vect', CountVectorizer(analyzer='word', stop_words=None,max_df=1.0)),
        ('tfid', TfidfTransformer()),
        ('clf', LogisticRegression())
])
scores = cross_val_score(pipeline_svm, texts, y, cv=5)
print("Classification score: %s (std %s)" % (np.mean(scores), np.std(scores)))

Classification score: 0.8545 (std 0.00509901951359279)


/home/jean/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Classification score: 0.8210000000000001 (std 0.004062019202317978)


# Conclusion

### Results on the all corpus

#### Naive Bayes:
Score de classification: 0.8099999999999999 (std 0.010839741694339374)

This could be seen as our baseline result.

#### Pipelines:

**classic:**
Classification score: %s (std %s) (0.812, 0.012884098726725092)

**tf-idf**
Classification score tf-idf: %s (std %s) (0.8125, 0.010488088481701477)

**sans mots fréquents**
Classification score sans mots fréquents: %s (std %s) (0.812, 0.012884098726725092)

**bigram**
Classification score bigram: %s (std %s) (0.8305, 0.009273618495495711)

**trigram**
Classification score trigram: %s (std %s) (0.8225, 0.017320508075688742)

**char**
Classification score char: %s (std %s) (0.6094999999999999, 0.018261982367749664)

We can see an improvement using the basic pipeline which is even better when we are getting rid of the most frequent words, seen as noise. 

The ponderation method "tf-idf" also help us to improve the scrore by having a better understanding of the corpus by assigning higher weights to the most frequent and relevant words. 

Using bigram or trigran seems to give higher result, this could be because it helped the program to have a better understanding of what the corpus is really about. Indeed, some words can have different meaning while being associated with other. *for example : If I take the word "extraordinary" alone, it might be seen as a positive word. However, if we have the association "extraordinary lame" it is in fact negative*. 
Also, taking too long associations might lead to overfitting because those associations will occur only in our training example.

Using character based classifier also seems a way less effective which is quite logical since the meaning of things and the way languages are build is more based on words.

#### Stemming:

Score de classification: 0.8099999999999999 (std 0.010839741694339374)

Stemming seems to give exactly the same result as the baseline, this could be explain by how it works. Indeed here we are associating each word to a root, it seems that we are not adding or taking off any information.

#### Part of speach:

Score de classification: 0.7825000000000001 (std 0.015411035007422467)

Using Part Of Speech technique, we are filtering the corpus to keep only the relevant information. It seems we filtered the corpus a bit too much here because we lost some information which leads to lower performance. 

#### Stop words:

Score de classification: 0.8035 (std 0.01617096162879622)

Same as before, lower score means we lost a bit of info. Here we probably get rid of too much words (stop words) while they were useful to get the meaning of the text. 

#### More complex classifier:

**LinearSVC**
Classification score: 0.8545 (std 0.00509901951359279)

**LogisticRegression**
Classification score: 0.8210000000000001 (std 0.004062019202317978)

Using more complex classifiers seems to help with the performances. By doing that, we are allowed to fit better the data (ie the corpus). 
